In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a7ef10d02bf0b6107676ca0b7efe97e25e89b38996c2a3297a2dd75dfbe2b5af
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import *

In [8]:
spark = SparkSession.builder.appName(name="testApp").getOrCreate()

In [9]:
spark

In [143]:
df1 = spark.read.csv("/content/drive/MyDrive/ColabTest/data/googleplaystore.csv",inferSchema=True, header=True)
df2 = spark.read.csv("/content/drive/MyDrive/ColabTest/data/googleplaystore_user_reviews.csv",inferSchema=True, header=True)
df3 = spark.read.csv("/content/drive/MyDrive/ColabTest/data/ab_data.csv",inferSchema=True, header=True)

In [144]:
print(df1.count())
df1.printSchema()

10841
root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [145]:
print(df2.count())
df2.printSchema()

64295
root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



In [146]:
print(df3.count())
df3.printSchema()

135920
root
 |-- user_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- group: string (nullable = true)
 |-- landing_page: string (nullable = true)
 |-- converted: integer (nullable = true)



In [151]:
df1.show(5)
# df1.orderBy(['Price'], ascending = [True]).show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [152]:
df2.show(1)

+--------------------+--------------------+--------------------+------------------+----------------------+
|                 App|   Translated_Review|           Sentiment|Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|          Positive|                   1.0|
+--------------------+--------------------+--------------------+------------------+----------------------+
only showing top 1 row



In [153]:
df3.show(1)

+-------+--------------------+-------+------------+---------+
|user_id|           timestamp|  group|landing_page|converted|
+-------+--------------------+-------+------------+---------+
| 851104|2017-01-21 22:11:...|control|    old_page|        0|
+-------+--------------------+-------+------------+---------+
only showing top 1 row



In [154]:
df1 = df1.drop("Size", "Content Rating", "Last Updated", "Android Ver", "Current Ver")

In [155]:
df1.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [156]:
df1.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [157]:
@udf
def replaceChar(ch1):
  return ch1.replace("+","").replace(",","").replace("$","")


In [158]:
df1 = df1.withColumn("Installs",replaceChar(col("Installs"))).withColumn("Price",replaceChar(col("Price")))

In [159]:
## Data cleaning

In [160]:
df1 = df1.withColumn("Rating", col("Rating").cast(FloatType())).withColumn("Reviews", col("reviews").cast(FloatType())).withColumn("Installs", col("Installs").cast(FloatType())).withColumn("Price", col("Price").cast(FloatType()))

In [161]:
df1.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: float (nullable = true)
 |-- Installs: float (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Genres: string (nullable = true)



In [162]:
df1.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|  159.0| 10000.0|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|  967.0|500000.0|Free|  0.0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [163]:
df1_view = df1.createOrReplaceTempView("View")

In [164]:
# Top 10 apps by review
spark.sql("Select App, sum(Reviews) from view group by 1 order by 2 desc limit 10").show()

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|2.66241988E8|
|  WhatsApp Messenger|2.07348296E8|
|      Clash of Clans|1.79558784E8|
|Messenger – Text ...|1.69932272E8|
|      Subway Surfers|1.66331958E8|
|    Candy Crush Saga|1.56993136E8|
|            Facebook|1.56286512E8|
|         8 Ball Pool| 9.9386198E7|
|        Clash Royale|   9.25303E7|
|            Snapchat|  6.804501E7|
+--------------------+------------+



In [165]:
# Top 10 apps by Installs all
spark.sql("Select App, sum(Installs) from view group by 1 order by 2 desc limit 10").show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|      Subway Surfers|        6.0E9|
|           Instagram|        4.0E9|
|            Hangouts|        4.0E9|
|        Google Drive|        4.0E9|
|         Google News|        4.0E9|
|       Google Photos|        4.0E9|
|    Candy Crush Saga|        3.5E9|
|  WhatsApp Messenger|        3.0E9|
|Messenger – Text ...|        3.0E9|
|Google Chrome: Fa...|        3.0E9|
+--------------------+-------------+



In [173]:
# Top 10 apps by Installs free
spark.sql("Select App, sum(Installs) from view where Type='Free' group by 1 order by 2 desc limit 10").show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|      Subway Surfers|        6.0E9|
|           Instagram|        4.0E9|
|            Hangouts|        4.0E9|
|        Google Drive|        4.0E9|
|         Google News|        4.0E9|
|       Google Photos|        4.0E9|
|    Candy Crush Saga|        3.5E9|
|  WhatsApp Messenger|        3.0E9|
|Messenger – Text ...|        3.0E9|
|Google Chrome: Fa...|        3.0E9|
+--------------------+-------------+



In [174]:
# Top 10 apps by Installs paid
spark.sql("Select App, sum(Installs) from view where Type='Paid' group by 1 order by 2 desc limit 10").show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|           Minecraft|        2.0E7|
|       Hitman Sniper|        1.0E7|
| Facetune - For Free|    3000000.0|
|Beautiful Widgets...|    2000000.0|
|          HD Widgets|    2000000.0|
|              Tasker|    1000000.0|
| Fruit Ninja Classic|    1000000.0|
|          True Skate|    1000000.0|
|   Where's My Water?|    1000000.0|
| DraStic DS Emulator|    1000000.0|
+--------------------+-------------+



In [166]:
# Top 5 installs by category
spark.sql("select Category, sum(Installs) from view group by 1  order by 2 desc limit 5").show()

+-------------+---------------+
|     Category|  sum(Installs)|
+-------------+---------------+
|         GAME|3.5086024415E10|
|COMMUNICATION|3.2647276251E10|
| PRODUCTIVITY|1.4176091369E10|
|       SOCIAL|1.4069867902E10|
|        TOOLS|1.1452771915E10|
+-------------+---------------+



In [170]:
# Top paid apps
spark.sql("select App, avg(Price) from view group by 1  order by 2 desc limit 5").show(truncate=False)

+------------------------+----------------+
|App                     |avg(Price)      |
+------------------------+----------------+
|I'm Rich - Trump Edition|400.0           |
|I Am Rich Premium       |399.989990234375|
|I am Rich               |399.989990234375|
|most expensive app (H)  |399.989990234375|
|I am Rich Plus          |399.989990234375|
+------------------------+----------------+



In [168]:
from os import truncate
df1.filter(df1["App"]=="Subway Surfers").show(truncate=False)

+--------------+--------+------+-----------+--------+----+-----+------+
|App           |Category|Rating|Reviews    |Installs|Type|Price|Genres|
+--------------+--------+------+-----------+--------+----+-----+------+
|Subway Surfers|GAME    |4.5   |2.7722264E7|1.0E9   |Free|0.0  |Arcade|
|Subway Surfers|GAME    |4.5   |2.7723192E7|1.0E9   |Free|0.0  |Arcade|
|Subway Surfers|GAME    |4.5   |2.7724094E7|1.0E9   |Free|0.0  |Arcade|
|Subway Surfers|GAME    |4.5   |2.7725352E7|1.0E9   |Free|0.0  |Arcade|
|Subway Surfers|GAME    |4.5   |2.7725352E7|1.0E9   |Free|0.0  |Arcade|
|Subway Surfers|GAME    |4.5   |2.7711704E7|1.0E9   |Free|0.0  |Arcade|
+--------------+--------+------+-----------+--------+----+-----+------+



In [169]:
df1.select("Installs").describe().show()

+-------+-------------------+
|summary|           Installs|
+-------+-------------------+
|  count|              10840|
|   mean|1.546378537782288E7|
| stddev|8.502944796373627E7|
|    min|                0.0|
|    max|              1.0E9|
+-------+-------------------+

